In [ ]:
import requests
from bs4 import BeautifulSoup
import http.cookiejar
import mechanize
import urllib

In [ ]:
url_login = "https://www.chicagocityscape.com/account.php"
url_auth = "https://www.chicagocityscape.com/account.php/account.php?provider=google&src="
session = SessionGoogle(url_login, url_auth, "reclaimfairelections@gmail.com", "rDxPK7p6W@5f")
base = session.get("https://www.chicagocityscape.com/ordinances.php#/?type=zoning&search_term=O2019-331")
soup = BeautifulSoup(base, 'html.parser')

In [152]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import csv
import time
import urllib

In [153]:
# Replace this with whatever topic page you'd like to scrape
ord_search_page = 'https://www.chicagocityscape.com/ordinances.php#/?type=zoning&search_term=O2019-331'

# Put your email and password in these variables, make sure to have the quotation marks around them
email = "reclaimfairelections@gmail.com"
pswd = "rDxPK7p6W@5f"


In [154]:
# Initialize webdriver
driver = webdriver.Firefox()
driver.get("https://www.chicagocityscape.com")
wait = WebDriverWait(driver, 30)

In [155]:
# Find sign-in by Google button and click it
elem = driver.find_element_by_partial_link_text('Sign in')
elem.click()
time.sleep(2)
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "fa-ul")))

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="371f6e20-1fd0-a849-8461-072e3ceb81f4", element="9a8288d4-41d5-9a49-a7fd-3fce98401551")>

In [156]:
goo = driver.find_element_by_partial_link_text('Google')
goo.click()
time.sleep(2)

In [157]:
# Enter Email address and submit
wait.until(EC.presence_of_element_located((By.ID, "headingText")))
email_input = driver.find_element_by_id("identifierId")
email_input.send_keys(email)
email_submit = driver.find_element_by_id("identifierNext").click()

In [158]:
wait.until(EC.presence_of_element_located((By.ID, "password")))

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="371f6e20-1fd0-a849-8461-072e3ceb81f4", element="339b1384-f2fc-7d41-be5b-c3d66e79598d")>

In [159]:
# Enter Password and submit
pw_input = driver.find_element_by_name("password")
pw_input.send_keys(pswd)
pw_submit = driver.find_element_by_id("passwordNext").click()
time.sleep(10)
# ADD WAIT HERE?

In [160]:
ord_page = 'https://www.chicagocityscape.com/ordinances.php'
driver.get(ord_page)
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "form-inline")))

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="371f6e20-1fd0-a849-8461-072e3ceb81f4", element="3a949efa-0b7a-a04e-9f3e-8dbb0849e5b9")>

In [161]:
ord_num = 'O2019-331'
ord_input = driver.find_element_by_id('search_term')
ord_input.send_keys(ord_num)
key = driver.find_element_by_class_name('btn')
key.click()

In [163]:
time.sleep(10)
wait.until(EC.presence_of_element_located((By.ID, "table_ordinances_wrapper")))
ord_link = driver.find_element_by_partial_link_text('Details')
ord_link.click()

In [164]:
# wait.until(EC.presence_of_element_located((By.ID, "")))
# get search page html to find ordinance id
driver.current_url
ord_details = driver.page_source
soup = BeautifulSoup(ord_details, "html5lib")


In [167]:
def find_ord_id(soup):
    link = ''
    for s in soup.find_all('a'):
        links = s.get('href')
        if links:
            if '/ordinances.php?ordinance=' in links:
                link = links
    return link.split('php')[1]

In [170]:
ord_code = find_ord_id(soup)

In [171]:
new_link = ord_page + ord_code
driver.get(new_link)

In [172]:
driver.current_url

'https://www.chicagocityscape.com/ordinances.php?ordinance=6588'

In [173]:
xx = driver.page_source
soup = BeautifulSoup(xx, "html5lib")
soup

<html class="fontawesome-i2svg-active fontawesome-i2svg-complete" lang="en"><head>
		<meta charset="utf-8"/>
	    <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
	    <meta content="width=device-width, initial-scale=1, user-scalable=no" name="viewport"/>
	    
	    <style type="text/css">svg:not(:root).svg-inline--fa{overflow:visible}.svg-inline--fa{display:inline-block;font-size:inherit;height:1em;overflow:visible;vertical-align:-.125em}.svg-inline--fa.fa-lg{vertical-align:-.225em}.svg-inline--fa.fa-w-1{width:.0625em}.svg-inline--fa.fa-w-2{width:.125em}.svg-inline--fa.fa-w-3{width:.1875em}.svg-inline--fa.fa-w-4{width:.25em}.svg-inline--fa.fa-w-5{width:.3125em}.svg-inline--fa.fa-w-6{width:.375em}.svg-inline--fa.fa-w-7{width:.4375em}.svg-inline--fa.fa-w-8{width:.5em}.svg-inline--fa.fa-w-9{width:.5625em}.svg-inline--fa.fa-w-10{width:.625em}.svg-inline--fa.fa-w-11{width:.6875em}.svg-inline--fa.fa-w-12{width:.75em}.svg-inline--fa.fa-w-13{width:.8125em}.svg-inline--fa.fa-w-14{width:.

In [213]:
ord_dict = {}

In [215]:
# get ordinance summary text; add it to dict
for s in soup.find_all('h3'):
    if s.next_element == 'Ordinance summary ':
        ord_dict['summary'] = s.next_sibling.next_sibling.text


In [216]:
ord_dict

{'summary': 'Construction of a residential development containing 152 dwelling units and 179 parking spaces, together with accessory and incidental uses.'}